In [ ]:
!git clone https://github.com/serengil/deepface.git
%cd deepface
!pip install -e .

In [ ]:
from google.colab import drive
import zipfile
import os

# Montar Google Drive
drive.mount('/content/drive')

# Rutas de tus archivos ZIP (modifica los nombres según los reales)
zips = [
    "/content/drive/MyDrive/TFG_Instagram/fotos_filtradas_LPGA.zip"
]

# Carpeta de destino base en /content
base_destino = "/content/"
os.makedirs(base_destino, exist_ok=True)

# Extraer cada ZIP en su propia subcarpeta
for zip_path in zips:
    nombre = os.path.splitext(os.path.basename(zip_path))[0]
    destino = os.path.join(base_destino, nombre)
    os.makedirs(destino, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(destino)
        print("Extraído")

In [ ]:
from PIL import Image
import os

# Ruta de la carpeta a analizar
carpeta = "/content/publicaciones_2024"  # cámbiala si es necesario

# Recorremos todos los archivos
for archivo in os.listdir(carpeta):
    if archivo.lower().endswith((".jpg")):
        ruta = os.path.join(carpeta, archivo)
        try:
            with Image.open(ruta) as img:
                img.verify()  # Verifica si la imagen es válida
        except Exception as e:
            print(f"Imagen corrupta eliminada: {archivo} → {e}")
            os.remove(ruta)

In [ ]:
import os
import pandas as pd
from deepface import DeepFace
import gc
from tqdm import tqdm

directorio_imagenes = "/content/fotos_filtradas_LPGA"
csv_salida = "/content/drive/MyDrive/TFG_Instagram/emociones_por_imagen_final_LPGA.csv"

resultados = []

# Filtrar imágenes válidas
imagenes = sorted([
    f for f in os.listdir(directorio_imagenes)
    if f.lower().endswith("_utc.jpg") or f.lower().endswith("_utc_1.jpg")
])

for nombre_imagen in tqdm(imagenes, desc="Procesando imágenes", unit="img"):
    ruta_imagen = os.path.join(directorio_imagenes, nombre_imagen)
    nombre_salida = nombre_imagen.replace("_UTC_1.jpg", "_UTC.jpg")

    try:
        resultados_rostros = DeepFace.analyze(
            img_path=ruta_imagen,
            actions=["emotion"],
            detector_backend="retinaface",
            enforce_detection=True
        )

        if isinstance(resultados_rostros, dict):
            resultados_rostros = [resultados_rostros]

        contador = {}
        for r in resultados_rostros:
            emocion = r.get("dominant_emotion")
            if emocion:
                contador[emocion] = contador.get(emocion, 0) + 1

        if contador:
            emocion_predominante = max(contador, key=contador.get)
            total_rostros = len(resultados_rostros)
        else:
            emocion_predominante = "neutral"
            total_rostros = 0

    except Exception as e:
        emocion_predominante = "neutral"
        total_rostros = 0

    resultados.append((nombre_salida, total_rostros, emocion_predominante))
    gc.collect()

# Guardar resultados
df_resultados = pd.DataFrame(resultados, columns=["imagen", "Rostros", "Emoción"])
df_resultados.to_csv(csv_salida, index=False, sep=';')
print(f"Resultados guardados en: {csv_salida}")

